In [0]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CustomerAnalysis").getOrCreate()
df = spark.read.csv("/Volumes/dbckwork/aibiwork/datafiles/customers-10000.csv", header=True, inferSchema=True)
df.createOrReplaceTempView("customer")

# Max country have word 'us'
df_select = df.select(
    col("First Name").alias("FirstName"),
    col("Last Name").alias("LastName"),
    "Country",
    "Email",
    "phone 1",
    "phone 2"
)
df_select = df_select.where(lower(df["Country"]).like("%us%")) \
            .groupBy("Country").count()
df_select = df_select.orderBy(desc("count"))

# Get all the list
df_all_list = df.select(
    col("First Name").alias("FirstName"),
    col("Last Name").alias("LastName"),
    "Country",
    "Email",
    "phone 1",
    "phone 2"
).where(df["Country"] == df_select.limit(1).collect()[0]["Country"])

# Check for is null email
df_null_contact = df.select(
    col("First Name").alias("FirstName"),
    col("Last Name").alias("LastName"),
    "Country",
    "Email",
    regexp_replace(col("Phone 1"), "x", "0").alias("Phone1"),
    "Phone 2"
).where(df["Phone 1"].like ("%x%"))
display(df_null_contact)